## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
<div style="width:200px;"> <img alt="" src="./fig/hw4q1_answer.jpg" width="800"> </div>

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

answer：
1. 证明：假设生成树T的最大边权小于某最小生成树（MST）的最大边权。MST移除最大边会分成两子树，T中有边连这两子树且边权小于MST最大边权。将此边加入MST并移除原最大边，会得到总权值更小的生成树，这与MST定义矛盾。所以，MST的最大边权在所有生成树的最大边权中是最小的，即MST的最大边权最小，可认为MST也是一种特殊的（最大边权最小的）生成树
2. 首先，过滤所有权重大于b的边；其次，检查剩余边构成的图是否连通；然后，使用BFS/DFS进行连通性判断

In [4]:
# add your code here
# algorithm of the liear time complexity 
def can_form_bst(edges, vertex_count, max_weight):
    adjacency_list = [[] for _ in range(vertex_count)]
    for node1, node2, weight in edges:
        if weight <= max_weight:
            adjacency_list[node1].append(node2)
            adjacency_list[node2].append(node1)
    visited = [False] * vertex_count
    queue = [0]
    visited[0] = True
    reachable_count = 1
    
    while queue:
        current_node = queue.pop(0)
        for neighbor in adjacency_list[current_node]:
            if not visited[neighbor]:
                visited[neighbor] = True
                reachable_count += 1
                queue.append(neighbor)
    
    return reachable_count == vertex_count

answer：
3. 首先，所有BST中最大边权的最小值等于MST的最大边权；然后，使用Kruskal算法找到MST，并记录其最大边权

In [3]:
def find_max_edge_in_mst(edges, vertex_count):
    sorted_edges = sorted(edges, key=lambda x: x[2])
    parent = list(range(vertex_count))
    
    def find(node):
        while parent[node] != node:
            parent[node] = parent[parent[node]]
            node = parent[node]
        return node
    
    max_edge_weight = 0
    edge_count = 0
    
    for node1, node2, weight in sorted_edges:
        root1 = find(node1)
        root2 = find(node2)
        if root1 != root2:
            parent[root2] = root1
            if weight > max_edge_weight:
                max_edge_weight = weight
            edge_count += 1
            if edge_count == vertex_count - 1:
                break
    
    return max_edge_weight

## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea：【求助了大模型AI】首先在预处理阶段，使用Dijkstra算法计算s到所有节点的最短路径d_s，以及构建反向图后计算t到所有节点的路径d_t_reversed。然后处理候选边，若原s到t可达，计算每条候选边s->u->v->t路径长度，找出使距离减少最大的边；若原不可达，则选择能连接s和t且路径最短的候选边。

时间复杂度：两次Dijkstra为(O(m + n log n))+遍历候选边（O(k)）

In [9]:
# 求助了大模型AI
# your algorithm time complexity is: (O(m + n log n))+(O(k))
import heapq

def dijkstra(graph, start, vertex_count):
    INF = float('inf')
    distances = [INF] * vertex_count
    distances[start] = 0
    heap = [(0, start)]
    
    while heap:
        current_distance, current_node = heapq.heappop(heap)
        if current_distance > distances[current_node]:
            continue
        for neighbor, weight in graph[current_node]:
            if distances[neighbor] > current_distance + weight:
                distances[neighbor] = current_distance + weight
                heapq.heappush(heap, (distances[neighbor], neighbor))
    
    return distances

def reverse_graph(original_graph, vertex_count):
    reversed_graph = [[] for _ in range(vertex_count)]
    for node in range(vertex_count):
        for neighbor, weight in original_graph[node]:
            reversed_graph[neighbor].append((node, weight))
    return reversed_graph

def find_best_new_road(original_graph, vertex_count, start, end, new_edges):
    dist_from_start = dijkstra(original_graph, start, vertex_count)
    
    reversed_graph = reverse_graph(original_graph, vertex_count)
    dist_from_end = dijkstra(reversed_graph, end, vertex_count)
    
    original_distance = dist_from_start[end]
    best_edge = None
    max_improvement = 0
    INF = float('inf')
    
    if original_distance < INF:
        for u, v, new_weight in new_edges:
            if dist_from_start[u] < INF and dist_from_end[v] < INF:
                candidate_dist = dist_from_start[u] + new_weight + dist_from_end[v]
                if candidate_dist < original_distance:
                    improvement = original_distance - candidate_dist
                    if improvement > max_improvement:
                        max_improvement = improvement
                        best_edge = (u, v, new_weight)
    else:
        min_candidate = INF
        for u, v, new_weight in new_edges:
            if dist_from_start[u] < INF and dist_from_end[v] < INF:
                candidate_dist = dist_from_start[u] + new_weight + dist_from_end[v]
                if candidate_dist < min_candidate:
                    min_candidate = candidate_dist
                    best_edge = (u, v, new_weight)
        if min_candidate < INF:
            max_improvement = INF  
    
    return best_edge, max_improvement


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：【做不来，使用了大模型AI】并查集（Union-Find）算法。首先，创建一个大小为 n²的并查集；然后将所有相邻的白色点连接起来；最后检查左上角和右下角是否属于同一个连通分量

时间复杂度：初始化并查集O(n²)+遍历网格连接点O(n²)+查找和合并操作O(1) = O(n²)

In [8]:
# 做不来，使用了大模型AI
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return False
        
        if self.rank[root_x] < self.rank[root_y]:
            root_x, root_y = root_y, root_x
        
        self.parent[root_y] = root_x
        
        if self.rank[root_x] == self.rank[root_y]:
            self.rank[root_x] += 1
        
        return True

def solve_escape_problem(n, black_points):
    def get_index(i, j):
        return i * n + j
    
    def is_valid(i, j):
        return 0 <= i < n and 0 <= j < n
    
    # 创建并查集
    uf = UnionFind(n * n)
    
    # 移动方向：上、下、左、右
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    # 将黑色点标记为不可通过
    black_set = set(black_points)
    
    # 连接相邻的白色点
    for i in range(n):
        for j in range(n):
            if (i, j) in black_set:
                continue
            
            for di, dj in directions:
                ni, nj = i + di, j + dj
                
                if is_valid(ni, nj) and (ni, nj) not in black_set:
                    uf.union(get_index(i, j), get_index(ni, nj))
    
    # 检查顶点是否连通
    top_left = get_index(0, 0)
    bottom_right = get_index(n-1, n-1)
    
    return uf.find(top_left) == uf.find(bottom_right)

# 测试用例
def test_escape_problem():
    # 图2中的黑色点坐标
    black_points = [
        (0, 1), (0, 4), (1, 1), (1, 3),
        (2, 0), (2, 2), (2, 4)
    ]
    
    n = 5  # 5x5的网格
    result = solve_escape_problem(n, black_points)
    print(f"是否存在逃脱路径: {result}")
test_escape_problem()

是否存在逃脱路径: False
